<a href="https://colab.research.google.com/github/geee05/Fine-Tuning-hugging-face-models/blob/main/rafalposwiata_deproberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#!pip install --upgrade accelerate

In [2]:
import torch
import numpy as np
import pandas as pd

!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes
import os

from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, AutoModelForSequenceClassification, Trainer

torch.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

'2.0.1+cu118'

In [3]:
dataset = pd.read_csv('/content/DAIC_train_3sp_sampled.csv')
tdataset= pd.read_csv('/content/DAIC_test_3sp.csv')

In [4]:
dataset['Group'].value_counts()

0    5132
1    5132
Name: Group, dtype: int64

In [5]:
dataset.rename(columns = {'Group':'label'}, inplace = True)

In [6]:
tdataset.rename(columns = {'Group':'label'}, inplace = True)

In [7]:
train = Dataset.from_pandas(dataset, preserve_index=False)
test = Dataset.from_pandas(tdataset, preserve_index=False)

In [8]:
print(train)
print(test)

Dataset({
    features: ['Unnamed: 0', 'path', 'text', 'label'],
    num_rows: 10264
})
Dataset({
    features: ['ID', 'path', 'text', 'label'],
    num_rows: 1914
})


In [9]:
tokenizer = AutoTokenizer.from_pretrained('paulagarciaserrano/roberta-depression-detection')

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_train_dataset = train.map(tokenize_function, batched=True)
tokenized_test_dataset = test.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/10264 [00:00<?, ? examples/s]

Map:   0%|          | 0/1914 [00:00<?, ? examples/s]

In [10]:
print(tokenized_train_dataset)
print(tokenized_test_dataset)

Dataset({
    features: ['Unnamed: 0', 'path', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10264
})
Dataset({
    features: ['ID', 'path', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1914
})


In [11]:
os.environ["WANDB_DISABLED"] = "true"

In [15]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch",num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500)   # default arguments for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained('paulagarciaserrano/roberta-depression-detection',ignore_mismatched_sizes=True, num_labels=2)  
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

trainer = Trainer(   # specifying trainer class
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()  # starts fine-tuning

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at paulagarciaserrano/roberta-depression-detection and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, 

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.631400,0.731655,0.648380,0.363292
2,0.292300,1.265119,0.666144,0.276331
3,0.145300,2.175715,0.654127,0.297240


<ipython-input-15-c30138164341>:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=3849, training_loss=0.37602565108289593, metrics={'train_runtime': 3141.0427, 'train_samples_per_second': 9.803, 'train_steps_per_second': 1.225, 'total_flos': 8101715616645120.0, 'train_loss': 0.37602565108289593, 'epoch': 3.0})